# 1. Load Python Libraries

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import shap

import sklearn
from sklearn import metrics
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, PowerTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import PredictionErrorDisplay

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

import keras_tuner as kt

import xgboost
from xgboost import XGBRegressor

# 2. Read dataset

In [ ]:
data = pd.read_excel('Data.xlsx')
data = data.filter(regex='^(?!Unnamed)')
data

# 3. Data Pre-processing

### Add Cellulose and Hemicellulose to Carbohydrate

In [ ]:
for i in range(data.shape[0]):
  if np.isnan(data.loc[i, 'Carbohydrates wt%']) and not np.isnan(data.loc[i, 'Cellulose wt%']) and not np.isnan(data.loc[i, 'Hemicellulose wt%']):
    data.loc[i, 'Carbohydrates wt%'] = data.loc[i, 'Cellulose wt%'] + data.loc[i, 'Hemicellulose wt%']
data[['Carbohydrates wt%', 'Cellulose wt%', 'Hemicellulose wt%']]

### Multiply Nitrogen by 6.25 to Obtain Protein

In [ ]:
# for i in range(data.shape[0]):
#   if data.loc[i, 'Feedstock Type'] in ['Woody Biomass', 'Algae', 'Food Waste', 'Household/Kitchen Waste'] and np.isnan(data.loc[i, 'Protein wt%']) and not np.isnan(data.loc[i, 'N%']):
#     data.loc[i, 'Protein wt%'] = data.loc[i, 'N%']*6.25
# data[['Protein wt%']]

### Replace Categorical Values with Numerical Values

In [ ]:
# Feedstock type:
# Woody Biomass = 0
# Algae = 1
# Model Compounds = 2
# Food Waste = 3
# Human Waste = 4
# Household/Kitchen Waste = 5
# Municipal Solid Waste = 6
# Animal Manure = 7
# Others = 8

# Pre-processing:
# No = 0
# Yes = 1

# Reactor Type:
# Batch/Autoclave = 0
# Plug Flow Reactor (PFR) = 1
# Continuous Stirred-Tank Reactor (CSTR) = 2

# Catalyst:
# No Catalyst = 0
# Uses Catalyst = 1

# Solvent:
# No Solvent = 0
# Uses Solvent = 1

data['Feedstock Type'].replace({'Woody Biomass': 0, 'Algae': 1, 'Model Compounds': 2, 'Food Waste': 3, 'Human Waste': 4, 'Household/Kitchen Waste': 5, 'Municipal Solid Waste': 6, 'Animal Manure': 7, 'Others': 8}, inplace = True)
data['Pre-processing'].replace({'No': 0, 'Yes': 1}, inplace = True)
data['Reactor Type'].replace({'Batch/Autoclave': 0, 'Plug Flow Reactor (PFR)': 1, 'Continuous Stirred-Tank Reactor (CSTR)': 2}, inplace = True)
data['Catalyst'] = data['Catalyst'].notna().astype(int)
data['Solvent'] = data['Solvent'].notna().astype(int)

data[['Feedstock Type', 'Pre-processing', 'Reactor Type', 'Catalyst', 'Solvent']]

### Drop Abnormal Datapoints

In [ ]:
print('Number of NaN biocrude values: ', data['Biocrude wt%'].isna().sum())
print('Number of "0" biocrude values: ', pd.Series(data['Biocrude wt%'] == 0).sum())

In [ ]:
data = data[data['Biocrude wt%'].notna()]
data = data[data['Biocrude wt%'] != 0]
data[['Biocrude wt%']]

### Show Missing Missing Percentages

In [ ]:
missing = data.isna().sum()/data.shape[0]*100
missing

### Drop Columns with Too Many Missing Values

In [ ]:
# Maintain original dataset
original_data = data

threshold = 0.3
min_count = int((1 - threshold) * len(data))
data = data.loc[:, data.notna().sum() >= min_count]
data

### Replace NaN with Mean

In [ ]:
imp = SimpleImputer()
imp.set_output(transform = 'pandas')
data = imp.fit_transform(data)
data

### Split into Input/Output Data

In [ ]:
# X = input = all columns up to (and including) Solvent
# Y = output = biocrude

X = data.loc[:, :'Solvent']
Y = data['Biocrude wt%']
X = X.to_numpy()
Y = Y.to_numpy().reshape(-1, 1)

### Split into Training/Testing/Validation Data

In [ ]:
# 60% train, 20% test, 20% validation
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 0)

print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape, '\n')

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

### Normalize Dataset

In [ ]:
X_normalizer = PowerTransformer()
Y_normalizer = PowerTransformer()

X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

# Yeo-Johnson Transformation
x_train = X_normalizer.fit_transform(x_train)
x_test = X_normalizer.fit_transform(x_test)
y_train = Y_normalizer.fit_transform(y_train)
y_test = Y_normalizer.fit_transform(y_test)

# Min Max Scaler
x_train = X_scaler.fit_transform(x_train)
x_test = X_scaler.fit_transform(x_test)
y_train = Y_scaler.fit_transform(y_train)
y_test = Y_scaler.fit_transform(y_test)

### Correlation Matrix

In [ ]:
matrix = data.corr(method = 'spearman')
cmap = sns.diverging_palette(250, 15, s = 75, l = 40, n = 9, center = 'light', as_cmap = True)
plt.figure(figsize = (12, 9))
ax = sns.heatmap(matrix, center = 0, annot = True, fmt = '.2f', square = True, cmap = cmap)

# 4. Build Tensorflow Model

### Compile Model

In [ ]:
keras.backend.clear_session()
model = Sequential()
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation = 'linear'))
learning_rate = 0.001
optimizer = Adam(learning_rate = learning_rate)
loss = 'mean_squared_error'
model.compile(optimizer = optimizer, loss = loss)

### Hyperparameter Tuning

In [ ]:
def model_builder(hp):
    model = Sequential()

    hp_units_1 = hp.Int('hp_units_1', min_value = 1, max_value = 128)
    hp_units_2 = hp.Int('hp_units_2', min_value = 1, max_value = 128)
    hp_units_3 = hp.Int('hp_units_3', min_value = 1, max_value = 128)
    hp_units_4 = hp.Int('hp_units_4', min_value = 1, max_value = 128)
    hp_units_5 = hp.Int('hp_units_5', min_value = 1, max_value = 128)
    hp_units_6 = hp.Int('hp_units_6', min_value = 1, max_value = 128)

    model.add(Dense(hp_units_1, activation = 'relu'))
    model.add(Dense(hp_units_2, activation = 'relu'))
    model.add(Dense(hp_units_3, activation = 'relu'))
    model.add(Dense(hp_units_4, activation = 'relu'))
    model.add(Dense(hp_units_5, activation = 'relu'))
    model.add(Dense(hp_units_6, activation = 'relu'))
    model.add(Dense(1, activation = 'linear'))

    hp_learning_rate = hp.Choice('hp_learning_rate', values = [0.01, 0.001, 0.0001])
    hp_loss = hp.Choice('hp_loss', values = ['mean_squared_error', 'mean_absolute_error', 'huber'])

    optimizer = Adam(learning_rate = hp_learning_rate)
    model.compile(optimizer = optimizer, loss = hp_loss)

    return model

In [ ]:
# tuner = kt.Hyperband(model_builder, objective = 'val_loss', max_epochs = 10, factor = 3, overwrite = True)
# stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True, start_from_epoch = 0)
# tuner.search(x_train, y_train, epochs = 200, validation_data = (x_val, y_val), callbacks = [stop_early])

In [ ]:
# best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
# print(best_hps.get('hp_units_1'))
# print(best_hps.get('hp_units_2'))
# print(best_hps.get('hp_units_3'))
# print(best_hps.get('hp_units_4'))
# print(best_hps.get('hp_units_5'))
# print(best_hps.get('hp_units_6'))
# print(best_hps.get('hp_learning_rate'))
# print(best_hps.get('hp_loss'))

In [ ]:
# model = tuner.hypermodel.build(best_hps)

### Train Model

In [ ]:
num_epochs = 200
history = model.fit(x_train, y_train, validation_split = 0.25, epochs = num_epochs)

### Test Model

In [ ]:
y_pred = model.predict(x_test)
print('Loss:\t\t', metrics.mean_squared_error(y_test, y_pred))
print('R^2 Score: \t', metrics.r2_score(y_test, y_pred))

### Perform Inverse Transformation of Output

In [ ]:
y_test_inverse = Y_scaler.inverse_transform(y_test)
y_test_inverse = Y_normalizer.inverse_transform(y_test_inverse)

y_pred_inverse = Y_scaler.inverse_transform(y_pred)
y_pred_inverse = Y_normalizer.inverse_transform(y_pred_inverse)

### Plot Losses

In [ ]:
plt.figure(figsize = (16, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss', fontsize = 20)
plt.ylabel('Loss', fontsize = 18)
plt.xlabel('Epoch', fontsize = 18)
plt.yticks(np.arange(0, 0.05, step = 0.01))
plt.xlim(0, num_epochs)
plt.ylim(0, 0.05)
plt.legend(['Training Loss', 'Validation Loss'], fontsize = 14, loc = 'upper right')
plt.show()

### Plot Biocrude Predicted vs Expected

In [ ]:
plt.figure(figsize = (16, 6))
plt.plot(y_test_inverse, color = "blue", label = 'Expected Output', marker = '.')
plt.plot(y_pred_inverse, color = "red", label = 'Predicted Output', marker = '.')
plt.legend(['Expected Output', 'Predicted Output'], fontsize = 12)
plt.xlabel("Data", fontsize = 20)
plt.ylabel("Biocrude Yield (%)", fontsize = 18)
plt.xlim(0, len(y_test))
plt.ylim(0, 100) 
plt.show()

### Plotting Cross-validation Predictions

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
PredictionErrorDisplay.from_predictions(y_test_inverse, y_pred=y_pred_inverse, kind="actual_vs_predicted", subsample=250, ax=axs[0], random_state=0)
axs[0].set_title("Actual vs. Predicted values")
PredictionErrorDisplay.from_predictions(y_test_inverse, y_pred=y_pred_inverse, kind="residual_vs_predicted", subsample=250, ax=axs[1], random_state=0)
axs[1].set_title("Residuals vs. Predicted Values")
fig.suptitle("Plotting cross-validated predictions")
plt.tight_layout()
plt.show()

### SHAP Explainer

In [ ]:
feature_names = data.loc[:, :'Solvent'].columns
explainer = shap.Explainer(model, x_train)
shap_values = explainer(x_test)
shap.summary_plot(shap_values, x_test, feature_names = feature_names)

# 5. Build XGBoost Model

### Build and Train Model

In [ ]:
xgb_model = XGBRegressor()
xgb_model.fit(x_train, y_train)
y_pred = xgb_model.predict(x_test).reshape(-1, 1)
xgb_model.score(x_test, y_test)

### Perform Inverse Transformation of Output

In [ ]:
y_test_inverse = Y_scaler.inverse_transform(y_test)
y_test_inverse = Y_normalizer.inverse_transform(y_test_inverse)

y_pred_inverse = Y_scaler.inverse_transform(y_pred)
y_pred_inverse = Y_normalizer.inverse_transform(y_pred_inverse)

### Plot Biocrude Predicted vs Expected

In [ ]:
plt.figure(figsize = (16, 6))
plt.plot(y_test_inverse, color = "blue", label = 'Expected Output', marker = '.')
plt.plot(y_pred_inverse, color = "red", label = 'Predicted Output', marker = '.')
plt.legend(['Expected Output', 'Predicted Output'], fontsize = 12)
plt.xlabel("Data", fontsize = 20)
plt.ylabel("Biocrude Yield (%)", fontsize = 18)
plt.xlim(0, len(y_test))
plt.ylim(0, 100)
plt.show()

### SHAP Explainer

In [ ]:
feature_names = data.loc[:, :'Solvent'].columns
explainer = shap.Explainer(xgb_model, x_train)
shap_values = explainer(x_test)
shap.summary_plot(shap_values, x_test, feature_names = feature_names)

# 6. Build Linear Regression Model

In [ ]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)
y_pred = linear_model.predict(x_test)
linear_model.score(x_test, y_test)

### Perform Inverse Transformation of Output

In [ ]:
y_test_inverse = Y_scaler.inverse_transform(y_test)
y_test_inverse = Y_normalizer.inverse_transform(y_test_inverse)

y_pred_inverse = Y_scaler.inverse_transform(y_pred)
y_pred_inverse = Y_normalizer.inverse_transform(y_pred_inverse)

### Plot Biocrude Predicted vs Expected

In [ ]:
plt.figure(figsize = (16, 6))
plt.plot(y_test_inverse, color = "blue", label = 'Expected Output', marker = '.')
plt.plot(y_pred_inverse, color = "red", label = 'Predicted Output', marker = '.')
plt.legend(['Expected Output', 'Predicted Output'], fontsize = 12)
plt.xlabel("Data", fontsize = 20)
plt.ylabel("Biocrude Yield (%)", fontsize = 18)
plt.xlim(0, len(y_test))
plt.ylim(0, 100)
plt.show()

### SHAP Explainer

In [ ]:
feature_names = data.loc[:, :'Solvent'].columns
explainer = shap.Explainer(linear_model, x_train)
shap_values = explainer(x_test)
shap.summary_plot(shap_values, x_test, feature_names = feature_names)

# 7. Linear Regression Models Using Carbohydrate, Protein, Lipid, and Ash

### Read dataset

In [ ]:
#Only including certain parameters
columns_to_include1 = ['Carbohydrates wt%', 'Protein wt%', 'Lipids wt%', 'Biocrude wt%']
columns_to_include2 = ['Carbohydrates wt%', 'Protein wt%', 'Lipids wt%', 'Ash wt%', 'Biocrude wt%']
data1 = original_data[columns_to_include1]
data2 = original_data[columns_to_include2]
data1 = data1.dropna()
data2 = data2.dropna()

### New pre-processing

In [ ]:
#Split into training and testing datasets
X_1 = data1.loc[:, :'Lipids wt%']
Y_1 = data1['Biocrude wt%']
X_1 = X_1.to_numpy()
Y_1 = Y_1.to_numpy().reshape(-1, 1)

#Split into training and testing datasets
X_2 = data2.loc[:, :'Ash wt%']
Y_2 = data2['Biocrude wt%']
X_2 = X_2.to_numpy()
Y_2 = Y_2.to_numpy().reshape(-1, 1)

In [ ]:
#Normalize datasets
x_train_1, x_test_1, y_train_1, y_test_1 = train_test_split(X_1, Y_1, test_size = 0.25)
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(X_2, Y_2, test_size = 0.25)

X_normalizer = PowerTransformer()
Y_normalizer = PowerTransformer()

X_scaler = MinMaxScaler()
Y_scaler = MinMaxScaler()

# Yeo-Johnson Transformation
#x_train = X_normalizer.fit_transform(x_train)
#x_test = X_normalizer.fit_transform(x_test)
#y_train = Y_normalizer.fit_transform(y_train)
#y_test = Y_normalizer.fit_transform(y_test)

# Min Max Scaler
x_train_1 = X_scaler.fit_transform(x_train_1)
x_test_1 = X_scaler.fit_transform(x_test_1)
y_train_1 = Y_scaler.fit_transform(y_train_1)
y_test_1 = Y_scaler.fit_transform(y_test_1)

x_train_2 = X_scaler.fit_transform(x_train_2)
x_test_2 = X_scaler.fit_transform(x_test_2)
y_train_2 = Y_scaler.fit_transform(y_train_2)
y_test_2 = Y_scaler.fit_transform(y_test_2)

### Output

In [ ]:
linear_model_1 = LinearRegression()
linear_model_1.fit(x_train_1, y_train_1)
y_pred_1 = linear_model_1.predict(x_test_1)
print('linear_model_1 score: ', linear_model_1.score(x_test_1, y_test_1))

linear_model_2 = LinearRegression()
linear_model_2.fit(x_train_2, y_train_2)
y_pred_2 = linear_model_2.predict(x_test_2)
print('linear_model_2 score: ', linear_model_2.score(x_test_2, y_test_2))